# Assignment 6

### Part A

In [185]:
import numpy as np
import pandas as pd

Consider a DataFrame giving information about the price of a stock in 1 year, and the probability of that price occurring.

In [186]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
                    
data

Price in 1 Year  Probability
0               32          0.3
1               43          0.4
2               54          0.3

Write the function ```exp_return```, which calculates the expected return of the stock, given the current stock price ```P```.

In [187]:
def exp_return(data, P):
    # Write your code here
    exp = 0
    for i in range(len(data.get('Probability'))):
        exp += ((data.get('Price in 1 Year')[i] / P) - 1) * data.get('Probability')[i]
    return exp

Write the function ```volatility```, which calculates the expected return of the stock, given the current stock price ```P```.

In [189]:
def volatility(data, P):
    exp_ret = exp_return(data, P)
    new_data = [(x / P) - exp_ret - 1.0 for x in data.get('Price in 1 Year')]
    new_data = [x ** 2 for x in new_data]
    # Write your code here
    variance = 0
    for i in range(len(data.get('Price in 1 Year'))):
        variance += new_data[i] *  data.get('Probability')[i]
    return np.sqrt(variance)

Write the function ```interval```, which calculates the 95% confidence interval for the expected return of the data. The corresponding Z score is 1.96. The function should return a list of size 2, with the first element representing the lower bound, and the second element representing the upper bound. 

In [193]:
def interval(data, P):
    # Write your code here
    se = volatility(data, P) / np.sqrt((data.get('Probability')).size)
    return [exp_return(data, P) - se * 1.96,
            exp_return(data, P) + se * 1.96]

Write the function ```exp_return_capm```, which, given the beta of a security ```beta```, the return on the market ```market_return```,  and the risk free interest rate ```rf```, calculates the expected return of the security.

In [ ]:
def exp_return_capm(beta, market_return, rf):
    return rf + beta * (market_return - rf)

### Part B

Write the function ```clean_data```, which will take a path to a csv file like the one provided. This function should remove all observations with a non number ```RET```, and remove all securities with (strictly) less than 500 observations. Your returned DataFrame should have 4 (non-index) columns, `PERMNO`, `date`, `CUSIP`, and `RET`.

In [ ]:
def clean_data(path):
    # Write your code here
    df = pd.read_csv(path)
    df = df[pd.to_numeric(df['RET'], errors='coerce').notnull()]
    grouped = df.groupby(['CUSIP']).filter(lambda x: len(x) >= 500)
    grouped.reset_index(drop=True, inplace=True)
    return grouped


Write the function ```avg_return```, which will calculate the average return for each security. This function should take in a DataFrame ```data```, formatted like one outputted by the function ```clean_data```. The results should be returned in a DataFrame formatted like the one supplied.

Additionally, write the function ```security_volatilities```, which does the same thing as ```avg_return```, except it returns the volatilities of the returns of the securities.

In [ ]:
def avg_return(data: pd.DataFrame):
    # Write your code here
    CUSIP = []
    Average_return = []
    data['RET'] = data['RET'].astype('float')

    for cusip, info in data.groupby('CUSIP'):
        CUSIP.append(cusip)
        Average_return.append(info['RET'].mean())
    
    output = pd.DataFrame(data = {
        "CUSIP" : CUSIP,
        "Average Return" : Average_return
    })
    
    return output

In [ ]:
def security_volatilities(data: pd.DataFrame):
    avg = avg_return(data)
    cusip = []
    volatility = []
    # Write your code here
    for id, groups in data.groupby('CUSIP'):
        variance = 1 / (len(groups) - 1)
        avg[avg['CUSIP'] == id]['Average Return']
        sum = groups['RET']
        sum = sum.apply(lambda x: (x - avg[avg['CUSIP'] == id]['Average Return']) ** 2)
        variance *= float((sum.sum()).iloc[0])
        cusip.append(id)
        volatility.append(np.sqrt(variance))
        

    output = pd.DataFrame(data = {
        "CUSIP" : cusip,
        "Volatility" : volatility
    })
    
    return output

Write the function ```correlation```, which will return the correlation between average return and volatility. This function takes two DataFrames, ```ret_data``` and ```vol_data```, which are the outputs of ```avg_return``` and ```security_volatilities``` respectively.

In [ ]:
def correlation(ret_data, vol_data):
    # Write your code here
    df = pd.merge(ret_data, vol_data, on='CUSIP', how='inner')
    df.set_index('CUSIP', inplace=True)
    return df.corr().iloc[0,1]